Related Resources:
- How to get GraphQL API Key: <https://bitquery.io/blog/graphql-with-python-javascript-and-ruby#Getting_API_Key>
- A tutorial for data query: <https://bitquery.io/blog/eth2-api#Daily_Blocks_and_Proposers_count>
- GraphQL Playground: <https://graphql.bitquery.io/ide>

# Part I Set-up

In [ ]:
!pip install -U kaleido #restart runtime after install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import datetime

import pandas as pd
import plotly.graph_objects as go
import requests
from plotly.subplots import make_subplots

In [ ]:
def run_query(query): 
    headers = {'X-API-KEY': 'BQYm4T9xFXiTsjWquq0ChBV18sC15UdY'}
    request = requests.post('https://graphql.bitquery.io/',
                            json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed and return code is {}. {}'.format(request.status_code,query))

# Part II Query Ethereum 2.0 Data

In [ ]:
query = """
{
  ethereum2 {
    deposits(options: {asc: "date.date"}) {
      date: date {
        date(format: "%Y-%m-%d")
      }
      count: count
      amount
    }
  }
}
"""

In [ ]:
result = run_query(query=query)

In [ ]:
deposit_history = result.get('data').get('ethereum2').get('deposits')

In [ ]:
deposit_history

[{'date': {'date': '2020-12-01'}, 'count': 6451, 'amount': 206401.0},
 {'date': {'date': '2020-12-02'}, 'count': 1644, 'amount': 52577.0},
 {'date': {'date': '2020-12-03'}, 'count': 1557, 'amount': 49824.0},
 {'date': {'date': '2020-12-04'}, 'count': 547, 'amount': 17504.0},
 {'date': {'date': '2020-12-05'}, 'count': 2574, 'amount': 82276.0},
 {'date': {'date': '2020-12-06'}, 'count': 1136, 'amount': 36263.0},
 {'date': {'date': '2020-12-07'}, 'count': 1194, 'amount': 38100.0},
 {'date': {'date': '2020-12-08'}, 'count': 1014, 'amount': 32448.0},
 {'date': {'date': '2020-12-09'}, 'count': 2900, 'amount': 92800.0},
 {'date': {'date': '2020-12-10'}, 'count': 1073, 'amount': 34336.0},
 {'date': {'date': '2020-12-11'}, 'count': 2062, 'amount': 65984.0},
 {'date': {'date': '2020-12-12'}, 'count': 1069, 'amount': 34208.0},
 {'date': {'date': '2020-12-13'}, 'count': 782, 'amount': 25024.0},
 {'date': {'date': '2020-12-14'}, 'count': 701, 'amount': 22432.0},
 {'date': {'date': '2020-12-15'}, 'c

# Part III Process and Visualize Data

In [ ]:
deposit_history_flatten = []

for item in deposit_history:
    date = item.get('date').get('date')
    count = item.get('count')
    amount = item.get('amount')

    deposit_history_flatten.append(
        dict(
            date=date,
            count=count,
            amount=amount
        )
    )

deposit_history_flatten

[{'date': '2020-12-01', 'count': 6451, 'amount': 206401.0},
 {'date': '2020-12-02', 'count': 1644, 'amount': 52577.0},
 {'date': '2020-12-03', 'count': 1557, 'amount': 49824.0},
 {'date': '2020-12-04', 'count': 547, 'amount': 17504.0},
 {'date': '2020-12-05', 'count': 2574, 'amount': 82276.0},
 {'date': '2020-12-06', 'count': 1136, 'amount': 36263.0},
 {'date': '2020-12-07', 'count': 1194, 'amount': 38100.0},
 {'date': '2020-12-08', 'count': 1014, 'amount': 32448.0},
 {'date': '2020-12-09', 'count': 2900, 'amount': 92800.0},
 {'date': '2020-12-10', 'count': 1073, 'amount': 34336.0},
 {'date': '2020-12-11', 'count': 2062, 'amount': 65984.0},
 {'date': '2020-12-12', 'count': 1069, 'amount': 34208.0},
 {'date': '2020-12-13', 'count': 782, 'amount': 25024.0},
 {'date': '2020-12-14', 'count': 701, 'amount': 22432.0},
 {'date': '2020-12-15', 'count': 791, 'amount': 25312.0},
 {'date': '2020-12-16', 'count': 495, 'amount': 15840.0},
 {'date': '2020-12-17', 'count': 640, 'amount': 20418.0},
 {

In [ ]:
deposit_history_df = pd.DataFrame(deposit_history_flatten)
deposit_history_df['date'] = deposit_history_df['date'].astype('datetime64[D]')

# aggregate
deposit_history_df['cumsum_amount'] = deposit_history_df['amount'].cumsum()

print(deposit_history_df.info())
display(deposit_history_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 826 entries, 0 to 825
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           826 non-null    datetime64[ns]
 1   count          826 non-null    int64         
 2   amount         826 non-null    float64       
 3   cumsum_amount  826 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 25.9 KB
None


,date,count,amount,cumsum_amount
0,2020-12-01,6451,206401.0,206401.0
1,2020-12-02,1644,52577.0,258978.0
2,2020-12-03,1557,49824.0,308802.0
3,2020-12-04,547,17504.0,326306.0
4,2020-12-05,2574,82276.0,408582.0
...,...,...,...,...
821,2023-03-02,643,19568.0,16763111.0
822,2023-03-03,1076,32512.0,16795623.0
823,2023-03-04,285,8272.0,16803895.0
824,2023-03-05,332,10112.0,16814007.0


In [ ]:
deposit_history_df.to_csv("Bitquery_Ethereum_deposit.csv")

In [ ]:
# some primary visualization
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)

fig.add_trace(
    go.Line(
        x=deposit_history_df['date'],
        y=deposit_history_df['amount'],
        name='Deposit Amount',
        line_color="blue"
    ),
    row=2, col=1
)

fig.add_trace(
    go.Line(
        x=deposit_history_df['date'],
        y=deposit_history_df['cumsum_amount'],
        name='Total Amount',
        line_color="red"
    ),
    row=1, col=1
)

fig.add_vline(
    x=datetime.datetime.strptime("2022-09-15", "%Y-%m-%d").timestamp() * 1000,
    annotation_position="right",
    annotation_text="The Merge: Sep 15, 2022",
    line_width=6,
    line_dash="dash", 
    line_color="purple"
)

fig.update_layout(dict(template='simple_white',
                  title='Staked Ether Time Series', width=800, height=600))

fig.show()

/usr/local/lib/python3.8/dist-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [ ]:
import plotly.io as pio
pio.write_image(fig, 'Bitquery_deposit.png')